In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [2]:
loader = CSVLoader(file_path="scraped_2511.csv")

data = loader.load()
data[0].page_content

": 0\nAll_content: ['', '', '', '', '', 'CHIL SEZ IT Park, Saravanampatty, Coimbatore - 035 info@dplus.co.in\\nEmployee login\\nHOME\\nABOUT US\\nSERVICES\\nFRANCHISE\\nCONTACT US\\nCALL US NOW\\n96 5 97 88888\\nWELCOME TO DPLUS CAR CARE\\nDplus is a One-Stop Solution for all your car servicing needs.\\nFREE ESTIMATE\\n01\\n02\\n03\\nWHO IS DPLUS CAR CARE?\\nAbout Our Company\\nA specialized team of experienced technicians thriving to help clients in their needs.\\nDplus Car Care is a leading brand in South India for Professional Car Washing ,Painting and Detailing Services Our services are supported by a team of well trained staffs who comes with a wealth of experience, not only in car washing and valeting professionalism and technical ability but also in customer service. our aim is to deliver the very best for our customers by providing an enhanced experience and an unequalled service.\\nOUR SERVICES\\nOur Service Plans To Meet All Your Demands\\nDplus Car Care is incredibly flexibl

In [3]:
type(data[0].metadata)

dict

In [4]:
import boto3
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from prompt import prompt
client = boto3.client("bedrock-runtime", region_name="us-east-1")
llm = ChatBedrock(client=client,model="mistral.mistral-small-2402-v1:0",temperature=0) #meta.llama3-1-70b-instruct-v1:0 mistral.mixtral-8x7b-instruct-v0:1 mistral.mistral-small-2402-v1:0

In [5]:
from langchain_groq import ChatGroq

groq_llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0,
)

In [6]:
llm.invoke("hi")

AIMessage(content=" Hello! How can I help you today? Is there something specific you would like to talk about or ask? I'm here to answer any questions you have to the best of my ability.", additional_kwargs={'usage': {'prompt_tokens': 5, 'completion_tokens': 40, 'total_tokens': 45}, 'stop_reason': None, 'model_id': 'mistral.mistral-small-2402-v1:0'}, response_metadata={'usage': {'prompt_tokens': 5, 'completion_tokens': 40, 'total_tokens': 45}, 'stop_reason': None, 'model_id': 'mistral.mistral-small-2402-v1:0'}, id='run-bcc670d6-267c-48ed-a155-d89eab1b77af-0', usage_metadata={'input_tokens': 5, 'output_tokens': 40, 'total_tokens': 45})

In [ ]:
#classify

def chat_with_llm(message):
    messages = [SystemMessage(content=prompt), HumanMessage(content=message)]
    response = llm.invoke(messages)
    return response

import re
def extract_json_from_string(s):
    json_pattern = re.compile(r'\{.*\}', re.DOTALL)
    
    # Search for the JSON pattern in the string
    match = json_pattern.search(s)
    
    if match:
        json_str = match.group(0)
        return json_str
    else:
        return None

In [8]:
data[4].page_content

': 15\nAll_content: []'

In [9]:
#trial
hi = chat_with_llm(data[4].page_content)
extract_json_from_string(hi.content)

'{\n  "business_name": "String",\n  "location_details": {\n    "full_address": "String",\n    "city": "String",\n    "state": "String",\n    "postal_code": "String",\n    "geo_coordinates": {\n      "latitude": "Float",\n      "longitude": "Float"\n    }\n  },\n  "contact_information": {\n    "primary_phone": "String",\n    "secondary_phone": "String",\n    "email": "String",\n    "website": "String"\n  },\n  "services": {\n    "primary_services": ["String", ...],\n    "secondary_services": ["String", ...],\n    "specialized_services": ["String", ...]\n  },\n  "operational_details": {\n    "working_hours": "String",\n    "days_of_operation": ["String", ...],\n    "established_year": "Integer"\n  },\n  "ratings_and_reviews": {\n    "average_rating": "Float",\n    "total_reviews": "Integer",\n    "platform_sources": ["String", ...]\n  },\n  "unique_selling_propositions": ["String", ...],\n  "additional_notes": "String"\n}'

In [ ]:
import time
import json

def json_converting(data):
    all_data = []

    # Loop over your data and call chat_with_llm
    for item in data:
        print(item.metadata)
        try:
            response = chat_with_llm(item.page_content)
            
            extracted_json_str = extract_json_from_string(response.content)
            
            if extracted_json_str:

                extracted_json = json.loads(extracted_json_str)
                extracted_json["metadata"] = item.metadata

                all_data.append(extracted_json)

                print(f"Successfully processed item")
            
            else:
                all_data.append(item.metadata)
            
        except Exception as e:
            print(f"Error processing item Error: {e}")
        
        # time.sleep(60)  # use sleep whenecer use groq freetrial

    with open("data2.json", "w") as json_file:
        json.dump(all_data, json_file, indent=4)

    print("Processing complete. Data stored in data.json.")

In [11]:
json_converting(data)

{'source': 'scraped_2511.csv', 'row': 0}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 1}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 2}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 3}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 4}
Error processing item Error: Expecting value: line 20 column 36 (char 443)
{'source': 'scraped_2511.csv', 'row': 5}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 6}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 7}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 8}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 9}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 10}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 11}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 12}
Successfully processed item
{'source': 'scraped_2511.csv', 'row': 13}
Successfull